In [9]:
import torch
from torch.distributions import *
from flows import *

# dataset is inverse transform sampling of arbitrary distribution
dim = 1
datapoints = 2500
# dist = Laplace(torch.tensor([0.0]), torch.tensor([1.]))
dist = Exponential(torch.tensor([1.0]))
# dist = Cauchy(torch.tensor([0.0]), torch.tensor([1.0]))
# dist = ContinuousBernoulli(torch.tensor([0.3]))
# dist = HalfCauchy(torch.tensor([1.0]))
# dist = HalfNormal(torch.tensor([1.0]))
# dist = Normal(torch.tensor([0.0]), torch.tensor([1.0]))
# dist = TransformedDistribution { ... }
# dist = Uniform(torch.tensor([0.0]), torch.tensor([1.0]))

table = dist.sample_n(datapoints)
dataset = torch.utils.data.TensorDataset(table)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1000)

In [ ]:
cov

In [10]:
# flows convert to normal dist
normal_flows = Flows(
    DenseTriangularFlow(dim, True),
    DenseTriangularFlow(dim, False),
    DenseTriangularFlow(dim, True),
    DenseTriangularFlow(dim, False),
    SoftlogFlow(),
    DenseTriangularFlow(dim, True),
    DenseTriangularFlow(dim, False),
    DenseTriangularFlow(dim, True),
    DenseTriangularFlow(dim, False),
    SoftlogFlow()
)

# final flow converts to normal

# train on NLL
net = FlowModule(normal_flows, NegLogLikelihoodLoss(dim))

optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)

In [11]:
final_loss = 0.0
for batch in range(200):  #1oop over the dataset multiple times
    for _, data in enumerate(dataloader, 0):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        loss = net(data[0])
        loss.backward()
        optimizer.step()
        final_loss = loss
print("final loss: %.03f" % final_loss)

final loss: 1.176


In [13]:
from numpy import arange

cdf_flows = Flows(
    normal_flows,
    NormalCdfFlow()
)

uniform = torch.tensor(list(arange(0.001, 0.999, 0.05))).unsqueeze(1)
# print(uniform.shape)
x = dist.icdf(uniform)
y, log_det = cdf_flows(x)

rmse = (y - uniform).pow(2).mean().sqrt()
print(rmse)

uniform_with_y = torch.stack([uniform, y], dim=1).squeeze(2)
print(uniform_with_y)

tensor(0.0470, grad_fn=<SqrtBackward>)
tensor([[0.0010, 0.0267],
        [0.0510, 0.0533],
        [0.1010, 0.0828],
        [0.1510, 0.1131],
        [0.2010, 0.1458],
        [0.2510, 0.1821],
        [0.3010, 0.2229],
        [0.3510, 0.2696],
        [0.4010, 0.3236],
        [0.4510, 0.3871],
        [0.5010, 0.4626],
        [0.5510, 0.5471],
        [0.6010, 0.6162],
        [0.6510, 0.6717],
        [0.7010, 0.7175],
        [0.7510, 0.7563],
        [0.8010, 0.7902],
        [0.8510, 0.8207],
        [0.9010, 0.8496],
        [0.9510, 0.8801]], grad_fn=<SqueezeBackward1>)


In [8]:
print(list(net.parameters()))

[Parameter containing:
tensor([[0.6278]], requires_grad=True), Parameter containing:
tensor([-0.3205], requires_grad=True), Parameter containing:
tensor([[0.6409]], requires_grad=True), Parameter containing:
tensor([-0.3354], requires_grad=True), Parameter containing:
tensor([[0.6872]], requires_grad=True), Parameter containing:
tensor([-0.3500], requires_grad=True), Parameter containing:
tensor([[0.7609]], requires_grad=True), Parameter containing:
tensor([-0.3631], requires_grad=True), Parameter containing:
tensor([[1.8049]], requires_grad=True), Parameter containing:
tensor([0.6451], requires_grad=True), Parameter containing:
tensor([[1.9340]], requires_grad=True), Parameter containing:
tensor([0.6000], requires_grad=True), Parameter containing:
tensor([[2.0000]], requires_grad=True), Parameter containing:
tensor([0.5487], requires_grad=True), Parameter containing:
tensor([[2.0259]], requires_grad=True), Parameter containing:
tensor([0.4914], requires_grad=True)]
